In [5]:
import os
import numpy as np
import rasterio
from PIL import Image
from rasterio.transform import Affine, array_bounds
from scipy.ndimage import uniform_filter
import re
from rasterio.warp import reproject, Resampling
import rasterio.crs
import json
from rasterio.errors import RasterioIOError
from pyproj import Transformer
import rasterio.windows # Added for robust tiling
import math # Added for math.floor, math.cos, math.radians
import shutil # Import the shutil module for directory operations


In [12]:
import numpy as np
import rasterio
from rasterio.transform import Affine, array_bounds
from scipy.ndimage import uniform_filter, label, sum_labels
import os
import re
from rasterio.warp import reproject, Resampling
import rasterio.crs
import json
from rasterio.errors import RasterioIOError
from pyproj import Transformer
import rasterio.windows
import math

def parse_uavsar_ann(ann_path):
    """
    Parses a UAVSAR .ann file to extract metadata for both MLC and GRD products.

    Args:
        ann_path (str): Path to the .ann file.

    Returns:
        dict: A dictionary containing 'mlc_meta' and 'grd_meta' dictionaries.
              Returns None if essential parsing fails.
    """
    mlc_meta = {}
    grd_meta = {}
    try:
        with open(ann_path, 'r') as f:
            content = f.read()

            def get_value(key, text, dtype=float):
                # Updated regex to more robustly capture float numbers, including scientific notation
                pattern = re.compile(r"{}\s*\(.*?\)\s*=\s*([-+]?\d*\.?\d+(?:[eE][-+]?\d*)?)".format(re.escape(key)))
                match = pattern.search(text)
                if match:
                    try:
                        return dtype(match.group(1))
                    except ValueError:
                        val_str = match.group(1)
                        if val_str.endswith('E') or val_str.endswith('e'):
                            val_str += '0' # Assume exponent is 0 if not specified
                            try:
                                return dtype(val_str)
                            except ValueError:
                                print(f"Warning: Could not convert '{val_str}' to {dtype} for key '{key}'.")
                                return None
                        print(f"Warning: Could not convert '{val_str}' to {dtype} for key '{key}'.")
                        return None
                return None
            
            def get_string_value(key, text):
                # Regex to capture string values for keys like "Look Direction"
                pattern = re.compile(r"{}\s*\(.*?\)\s*=\s*([a-zA-Z\s]+)".format(re.escape(key)))
                match = pattern.search(text)
                if match:
                    return match.group(1).strip()
                return None

            # --- Extract MLC Dimensions ---
            mlc_meta['width'] = get_value('mlc_pwr.set_cols', content, dtype=int)
            mlc_meta['height'] = get_value('mlc_pwr.set_rows', content, dtype=int)
            # Also get MLC pixel sizes and origin for its source transform
            mlc_meta['col_addr'] = get_value('mlc_pwr.col_addr', content)
            mlc_meta['row_addr'] = get_value('mlc_pwr.row_addr', content)
            mlc_meta['col_mult'] = get_value('mlc_pwr.col_mult', content)
            mlc_meta['row_mult'] = get_value('mlc_pwr.row_mult', content)

            if any(v is None for v in [mlc_meta['width'], mlc_meta['height'],
                                       mlc_meta['col_addr'], mlc_meta['row_addr'],
                                       mlc_meta['col_mult'], mlc_meta['row_mult']]):
                print("Error: Could not parse essential MLC dimensions or georeferencing from .ann file.")
                return None

            # --- Extract GRD Georeferencing Information ---
            grd_ul_x = get_value('grd_pwr.col_addr', content)
            grd_ul_y = get_value('grd_pwr.row_addr', content)
            grd_pixel_size_x = get_value('grd_pwr.col_mult', content)
            grd_pixel_size_y = get_value('grd_pwr.row_mult', content)
            
            grd_col_mult_y = get_value('grd_pwr.col_mult_y', content)
            if grd_col_mult_y is None: grd_col_mult_y = 0.0
            
            grd_row_mult_x = get_value('grd_pwr.row_mult_x', content)
            if grd_row_mult_x is not None:
                grd_row_mult_x = grd_row_mult_x
            else:
                grd_row_mult_x = 0.0


            grd_meta['width'] = get_value('grd_pwr.set_cols', content, dtype=int)
            grd_meta['height'] = get_value('grd_pwr.set_rows', content, dtype=int)

            if any(v is None for v in [grd_ul_x, grd_ul_y, grd_pixel_size_x, grd_pixel_size_y, grd_meta['width'], grd_meta['height']]):
                 print("Error: Could not parse essential GRD georeferencing parameters or dimensions from .ann file.")
                 return None
            
            grd_meta['transform'] = Affine(grd_pixel_size_x, grd_col_mult_y, grd_ul_x,
                                           grd_row_mult_x, grd_pixel_size_y, grd_ul_y)
            grd_meta['crs'] = 'EPSG:4326' # GRD data is in Lat/Lon

            # Get Peg Lat/Lon from ANN file for centering the custom MLC CRS
            mlc_meta['peg_lat'] = get_value('Peg Latitude', content)
            mlc_meta['peg_lon'] = get_value('Peg Longitude', content)
            mlc_meta['peg_heading'] = get_value('Peg Heading', content) # Get Peg Heading
            mlc_meta['look_direction'] = get_string_value('Look Direction', content) # Get look direction
            
            if any(v is None for v in [mlc_meta['peg_lat'], mlc_meta['peg_lon'], mlc_meta['peg_heading'], mlc_meta['look_direction']]):
                print("Error: Could not get Peg Latitude/Longitude/Heading/Look Direction from .ann file.")
                return None


            print("Successfully parsed .ann file.")
            print(f"  - MLC Dimensions: {mlc_meta['width']} x {mlc_meta['height']}")
            print(f"  - GRD Dimensions: {grd_meta['width']} x {grd_meta['height']}")
            print(f"  - GRD Upper-left corner: ({grd_ul_x}, {grd_ul_y})")
            print(f"  - GRD Pixel size (dx, dy): ({grd_pixel_size_x}, {grd_pixel_size_y})")
            print(f"  - GRD Shear/Rotation (dx_y, dy_x): ({grd_col_mult_y}, {grd_row_mult_x})")
            print(f"  - GRD CRS: {grd_meta['crs']}")
            print(f"  - Generated GRD Affine Transform: {grd_meta['transform']}")
            
            left, bottom, right, top = array_bounds(grd_meta['height'], grd_meta['width'], grd_meta['transform'])
            print(f"  - Calculated GRD Bounding Box: Left={left}, Bottom={bottom}, Right={right}, Top={top}")
            print(f"  - GRD Latitude Range: [{bottom}, {top}]")

            if bottom < -90 or top > 90:
                print("Warning: Calculated GRD latitude range is outside [-90, 90]. This might cause issues in GEE.")

            return {'mlc_meta': mlc_meta, 'grd_meta': grd_meta}

    except FileNotFoundError:
        print(f"Error: Annotation file not found at {ann_path}")
        return None
    except Exception as e:
        print(f"An error occurred while parsing the .ann file: {e}")
        return None


def build_mlc_covariance_matrix(base_path, rows, cols):
    """
    Constructs a 3x3 covariance matrix from UAVSAR .mlc files.
    """
    print("\nReading multi-look complex (.mlc) files...")
    file_suffixes = {
        'C11': 'HHHH_CX_01.mlc',
        'C22': 'HVHV_CX_01.mlc',
        'C33': 'VVVV_CX_01.mlc',
        'C12': 'HHHV_CX_01.mlc',
        'C13': 'HHVV_CX_01.mlc',
        'C23': 'HVVV_CX_01.mlc'
    }

    dtypes = {
        'C11': np.dtype('<f4'),
        'C22': np.dtype('<f4'),
        'C33': np.dtype('<f4'),
        'C12': np.dtype('<c8'),
        'C13': np.dtype('<c8'),
        'C23': np.dtype('<c8')
    }

    components = {}
    for key, suffix in file_suffixes.items():
        filepath = f"{base_path}{suffix}" 
        print(f"  - Reading {filepath}")
        if not os.path.exists(filepath):
            print(f"Error: File not found at {filepath}. Please ensure all necessary MLC files are present.")
            return None
        
        data = np.fromfile(filepath, dtype=dtypes[key])
        
        expected_size = rows * cols
        if data.size != expected_size:
            print(f"Error: Mismatch in expected data size for {filepath}.")
            print(f"  Expected {expected_size} values of type {dtypes[key]}, but got {data.size}.")
            print(f"  This usually means the parsed dimensions ({rows}x{cols}) from .ann are incorrect or the file format is not as expected.")
            return None

        components[key] = data.reshape((rows, cols))

    print("\nAssembling the 3x3 covariance matrix for each pixel...")
    C3 = np.zeros((rows, cols, 3, 3), dtype=np.complex64)

    C3[:, :, 0, 0] = components['C11']
    C3[:, :, 1, 1] = components['C22']
    C3[:, :, 2, 2] = components['C33']

    C3[:, :, 0, 1] = components['C12']
    C3[:, :, 0, 2] = components['C13']
    C3[:, :, 1, 2] = np.conjugate(C3[:, :, 0, 1]) # C21 = C12*
    C3[:, :, 2, 0] = np.conjugate(C3[:, :, 0, 2]) # C31 = C13*
    C3[:, :, 2, 1] = np.conjugate(C3[:, :, 1, 2]) # C32 = C23*

    print("Assembly complete.")
    return C3

def refined_lee(image, size=7):
    """
    Applies a Refined Lee filter to a single-band image.
    """
    mean = uniform_filter(image, size, mode='reflect')
    mean_sq = uniform_filter(image**2, size, mode='reflect')
    var = mean_sq - mean**2
    noise_var = np.mean(var)
    weights = var / (var + noise_var + 1e-8)
    weights = np.clip(weights, 0, 1)
    return np.clip(mean + weights * (image - mean), 0, None)

def normalize_and_scale(arr):
    """
    Normalizes array values to the 99.5th percentile and scales to [0, 255].
    """
    if np.iscomplexobj(arr):
        arr_real = np.abs(arr)
    else:
        arr_real = arr
    
    arr_real = arr_real.astype(np.float32)
    max_val = np.nanpercentile(arr_real, 99.5)
    
    if max_val <= 0:
        max_val = np.nanmax(arr_real)
    if max_val <= 0:
        return np.zeros_like(arr_real, dtype=np.uint8)

    scaled = np.clip(arr_real / max_val, 0, 1) * 255
    return scaled.astype(np.uint8)

def get_existing_tile_ids(folder_path):
    """
    Extracts tile IDs from filenames in a given folder.
    Assumes filenames are in the format 'tile_ID.tif'.

    Args:
        folder_path (str): Path to the folder containing tile files.

    Returns:
        set: A set of integer tile IDs found in the folder.
    """
    tile_ids = set()
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            match = re.match(r'tile_(\d+)\.tif', filename)
            if match:
                tile_ids.add(int(match.group(1)))
    return tile_ids

def delete_tiles_by_ids(folder_path, ids_to_delete):
    """
    Deletes GeoTIFF tiles from a specified folder based on a set of tile IDs.

    Args:
        folder_path (str): Path to the folder containing tile files.
        ids_to_delete (set): A set of integer tile IDs to be deleted.
    """
    print(f"\nAttempting to delete {len(ids_to_delete)} tiles from '{folder_path}'...")
    deleted_count = 0
    if os.path.exists(folder_path):
        for tile_id in ids_to_delete:
            filename = f"tile_{tile_id}.tif"
            filepath = os.path.join(folder_path, filename)
            if os.path.exists(filepath):
                try:
                    os.remove(filepath)
                    print(f"  Deleted: {filepath}")
                    deleted_count += 1
                except OSError as e:
                    print(f"  Error deleting {filepath}: {e}")
            # else: # Optional: print a warning if tile not found for deletion
            #    print(f"  Warning: Tile {filename} not found in {folder_path} for deletion.")
    print(f"Finished deleting tiles from '{folder_path}'. Total deleted: {deleted_count}")


def export_tiled_image_local(input_tif_path, export_folder, roi_bbox, tile_grid_size_meters, output_tile_pixel_size=(256, 256), allowed_tile_ids=None, collect_skipped_zero_ids=False):
    """
    Tiles a GeoTIFF image into smaller GeoTIFFs based on a geographic ROI and meter-based grid,
    and exports them to a local folder. Skips tiles where every pixel value is zero,
    and an optional set of allowed tile IDs.

    Args:
        input_tif_path (str): Path to the folder containing input GeoTIFF files.
        export_folder (str): Path to the folder where PNG files will be saved.
        roi_bbox (list): [min_lon, min_lat, max_lon, max_lat] defining the region of interest.
        tile_grid_size_meters (float): The desired side length of each grid cell in meters.
        output_tile_pixel_size (tuple): Desired (height, width) of each output tile in pixels.
        allowed_tile_ids (set, optional): A set of tile IDs. If provided, only tiles whose ID
                                          is in this set will be processed. Defaults to None (all tiles allowed).
        collect_skipped_zero_ids (bool): If True, returns a list of tile IDs that were skipped
                                         because all their pixels were zero.
    Returns:
        dict: A dictionary of tiling statistics, optionally including 'skipped_zero_ids_list'.
    """
    print(f"\nStarting tiling process for {input_tif_path} over ROI {roi_bbox}...")
    
    os.makedirs(export_folder, exist_ok=True)

    min_lon, min_lat, max_lon, max_lat = roi_bbox
    output_tile_height_pixels, output_tile_width_pixels = output_tile_pixel_size

    with rasterio.open(input_tif_path) as src:
        src_profile = src.profile
        src_crs = src.crs
        src_transform = src.transform

        # Determine UTM zone for the center of the ROI to perform meter-based calculations
        # The ROI is in Lat/Lon (EPSG:4326), so derive UTM zone from its center.
        center_lat_roi = (min_lat + max_lat) / 2
        center_lon_roi = (min_lon + max_lon) / 2
        
        # Calculate UTM zone number (1-60)
        utm_zone_number = int(math.floor((center_lon_roi + 180) / 6) + 1)
        # Determine if Northern or Southern hemisphere
        utm_hemisphere = '326' if center_lat_roi >= 0 else '327'
        utm_epsg_code = int(f"{utm_hemisphere}{utm_zone_number}")
        
        print(f"  Using UTM CRS: EPSG:{utm_epsg_code} for meter-based grid calculation.")
        
        # This transformer converts the ROI (which is in Lat/Lon, EPSG:4326) to the target UTM for grid calculation
        transformer_roi_ll_to_utm = Transformer.from_crs("EPSG:4326", f"EPSG:{utm_epsg_code}", always_xy=True)
        # This transformer converts the UTM grid back to the source file's CRS (src_crs) for window calculation
        transformer_utm_to_src_crs = Transformer.from_crs(f"EPSG:{utm_epsg_code}", src_crs, always_xy=True)


        # Convert ROI bounds (which are in Lat/Lon, EPSG:4326) to UTM
        utm_e_min, utm_n_min = transformer_roi_ll_to_utm.transform(min_lon, min_lat)
        utm_e_max, utm_n_max = transformer_roi_ll_to_utm.transform(max_lon, max_lat)

        # Ensure min/max are correct after transform (due to potential wrapping or negative coords)
        utm_e_min, utm_e_max = min(utm_e_min, utm_e_max), max(utm_e_min, utm_e_max)
        utm_n_min, utm_n_max = min(utm_n_min, utm_n_max), max(utm_n_min, utm_n_max)

        # Calculate number of tiles in the UTM grid
        num_tiles_utm_x = int(np.ceil((utm_e_max - utm_e_min) / tile_grid_size_meters))
        num_tiles_utm_y = int(np.ceil((utm_n_max - utm_n_min) / tile_grid_size_meters))
        
        print(f"  Calculated UTM grid dimensions: {num_tiles_utm_x} columns x {num_tiles_utm_y} rows.")
        total_grid_cells = num_tiles_utm_x * num_tiles_utm_y
        print(f"  Total potential tiles in grid: {total_grid_cells}")

        tile_id_counter = 0 # Use a separate counter for iteration
        skipped_zero_dim_tiles = 0
        skipped_all_zero_pixels_tiles = 0 
        skipped_by_allowed_list = 0 
        exported_tiles_count = 0
        skipped_all_zero_pixels_ids_list = [] if collect_skipped_zero_ids else None


        for utm_row_idx in range(num_tiles_utm_y):
            for utm_col_idx in range(num_tiles_utm_x):
                # Check if current tile_id_counter is in the allowed_tile_ids list
                if allowed_tile_ids is not None and tile_id_counter not in allowed_tile_ids:
                    print(f"  Skipping tile_{tile_id_counter} (not in allowed_tile_ids list).")
                    skipped_by_allowed_list += 1
                    tile_id_counter += 1
                    continue

                # Calculate UTM bounds of the current grid cell
                current_utm_e_min = utm_e_min + utm_col_idx * tile_grid_size_meters
                current_utm_n_min = utm_n_min + utm_row_idx * tile_grid_size_meters
                current_utm_e_max = current_utm_e_min + tile_grid_size_meters
                current_utm_n_max = current_utm_n_min + tile_grid_size_meters

                # Convert current UTM grid cell bounds back to the SRC_CRS of the input TIFF
                tile_left_src_crs, tile_bottom_src_crs = transformer_utm_to_src_crs.transform(current_utm_e_min, current_utm_n_min)
                tile_right_src_crs, tile_top_src_crs = transformer_utm_to_src_crs.transform(current_utm_e_max, current_utm_n_max)

                # Ensure correct order for rasterio.window (left, bottom, right, top)
                tile_left = min(tile_left_src_crs, tile_right_src_crs)
                tile_right = max(tile_left_src_crs, tile_right_src_crs)
                tile_bottom = min(tile_bottom_src_crs, tile_top_src_crs)
                tile_top = max(tile_bottom_src_crs, tile_top_src_crs)

                # Get the window in the source image that covers this geographic tile
                try:
                    # Define a window that might extend beyond the source image, then clip it
                    unclipped_window = rasterio.windows.from_bounds(
                        tile_left, tile_bottom, tile_right, tile_top, src.transform
                    )
                    
                    # Clip the window to the source image's bounds
                    window = unclipped_window.intersection(rasterio.windows.Window(0, 0, src.width, src.height))

                    if window.width <= 0 or window.height <= 0:
                        print(f"  Skipping tile_{tile_id_counter} (window has zero or negative dimensions, likely outside image).")
                        skipped_zero_dim_tiles += 1
                        tile_id_counter += 1
                        continue

                    # Read data from the source image using the clipped window
                    extracted_data = src.read(window=window, boundless=True)

                except Exception as e:
                    print(f"  Error getting window or reading data for tile_{tile_id_counter}: {e}")
                    skipped_zero_dim_tiles += 1 
                    tile_id_counter += 1
                    continue

                # --- Skipping Logic ---
                # Skip if the sum of all pixels is 0 (all black)
                if np.sum(extracted_data) == 0:
                    print(f"  Skipping tile_{tile_id_counter} (all zero pixels).")
                    skipped_all_zero_pixels_tiles += 1
                    if collect_skipped_zero_ids:
                        skipped_all_zero_pixels_ids_list.append(tile_id_counter)
                    tile_id_counter += 1
                    continue
                # --- End Skipping Logic ---
                
                # Define the destination transform for the output tile (fixed 256x256 pixels)
                # The pixel size in SRC_CRS units for the output tile
                dst_pixel_x_size = (tile_right - tile_left) / output_tile_width_pixels
                dst_pixel_y_size = (tile_bottom - tile_top) / output_tile_height_pixels # Will be negative for north-up

                dst_transform = Affine(dst_pixel_x_size, 0, tile_left,
                                       0, dst_pixel_y_size, tile_top)

                # Prepare the destination array for reprojection
                output_array = np.zeros(
                    (src_profile['count'], output_tile_height_pixels, output_tile_width_pixels),
                    dtype=src_profile['dtype'] # Use original data type for reprojection
                )

                # Reproject the extracted data to the fixed output tile size and resolution
                try:
                    reproject(
                        source=extracted_data,
                        destination=output_array,
                        src_transform=src.window_transform(window),
                        src_crs=src_crs,
                        dst_transform=dst_transform,
                        dst_crs=src_crs, # Output CRS is the same as input's src_crs
                        resampling=Resampling.bilinear,
                        num_threads=os.cpu_count() # Use all available CPU cores for faster reprojection
                    )
                except Exception as e:
                    print(f"  Error during reprojection of tile_{tile_id_counter}: {e}")
                    skipped_zero_dim_tiles += 1 
                    tile_id_counter += 1
                    continue

                # Update profile for the tile
                tile_profile = src_profile.copy()
                tile_profile.update({
                    'height': output_tile_pixel_size[0], 
                    'width': output_tile_pixel_size[1],  
                    'transform': dst_transform,
                    'compress': 'lzw',
                    'dtype': output_array.dtype 
                })

                output_filepath = os.path.join(export_folder, f'tile_{tile_id_counter}.tif')
                with rasterio.open(output_filepath, 'w', **tile_profile) as dst:
                    dst.write(output_array)
                print(f"  Exported tile: {output_filepath}")
                exported_tiles_count += 1
                tile_id_counter += 1
    
    print(f"\nFinished tiling process for {input_tif_path}.")
    print(f"  Total grid cells considered: {total_grid_cells}")
    print(f"  Tiles skipped (window/read error): {skipped_zero_dim_tiles}")
    print(f"  Tiles skipped (all zero pixels): {skipped_all_zero_pixels_tiles}") 
    print(f"  Tiles skipped (not in allowed list): {skipped_by_allowed_list}")
    print(f"  Successfully exported tiles: {exported_tiles_count}")

    stats = {
        "total_grid_cells": total_grid_cells,
        "skipped_zero_dim_tiles": skipped_zero_dim_tiles,
        "skipped_all_zero_pixels_tiles": skipped_all_zero_pixels_tiles,
        "skipped_by_allowed_list": skipped_by_allowed_list,
        "exported_tiles_count": exported_tiles_count
    }
    if collect_skipped_zero_ids:
        stats["skipped_zero_ids_list"] = skipped_all_zero_pixels_ids_list
    return stats


def main():
    # === Step 1: User Inputs ===
    DATA_DIRECTORY = '.' 
    BASE_FILENAME = 'cpfear_13510_18065_008_180918_L090_CX_01_mlc/cpfear_13510_18065_008_180918_L090' 
    
    ann_filename = f"{BASE_FILENAME}_CX_01.ann"
    ann_path = os.path.join(DATA_DIRECTORY, ann_filename)
    full_base_path = os.path.join(DATA_DIRECTORY, BASE_FILENAME)

    freeman_output = "freeman_rgb_mlc_reprojected_lee.tif"
    pauli_output = "pauli_rgb_mlc_reprojected_lee.tif"

    # === Step 2: Parse .ann file for metadata ===
    all_meta = parse_uavsar_ann(ann_path)
    if not all_meta:
        print("Exiting due to parsing error.")
        return
    
    mlc_width, mlc_height = all_meta['mlc_meta']['width'], all_meta['mlc_meta']['height']
    mlc_col_addr = all_meta['mlc_meta']['col_addr']
    mlc_row_addr = all_meta['mlc_meta']['row_addr']
    mlc_px_size_x = all_meta['mlc_meta']['col_mult']
    mlc_px_size_y = all_meta['mlc_meta']['row_mult']
    mlc_peg_lat = all_meta['mlc_meta']['peg_lat']
    mlc_peg_lon = all_meta['mlc_meta']['peg_lon']
    mlc_peg_heading = all_meta['mlc_meta']['peg_heading']
    mlc_look_direction = all_meta['mlc_meta']['look_direction']

    grd_width, grd_height = all_meta['grd_meta']['width'], all_meta['grd_meta']['height'] 
    grd_transform = all_meta['grd_meta']['transform']
    grd_crs = all_meta['grd_meta']['crs']

    # === Step 3: Build Covariance Matrix from .mlc files ===
    C3 = build_mlc_covariance_matrix(full_base_path, mlc_height, mlc_width)
    if C3 is None:
        print("Exiting due to error in building covariance matrix.")
        return

    # === Step 4: Extract Covariance Matrix Elements ===
    print("\nExtracting Cij components from the assembled covariance matrix...")
    C11 = C3[:, :, 0, 0].real
    C22 = C3[:, :, 1, 1].real
    C33 = C3[:, :, 2, 2].real
    C12 = C3[:, :, 0, 1]
    C13 = C3[:, :, 0, 2]
    C23 = C3[:, :, 1, 2]

    # === Step 5: Freeman–Durden Decomposition ===
    print("Performing Freeman-Durden decomposition (Surface, Double-bounce, Volume scattering)...")
    Ps = (C11 + C33 - (C13 + np.conjugate(C13))).real
    Pd = (C11 + C33 + (C13 + np.conjugate(C13))).real
    Pv = 4.0 * C22
    
    # === Step 6: Pauli Decomposition ===
    print("Performing Pauli decomposition...")
    Pauli_R = np.abs(C11 - C33)
    Pauli_G = 2 * np.abs(C12)
    Pauli_B = np.abs(C11 + C33)

    # For reprojection, use the full arrays
    Ps_to_reproject = Ps
    Pd_to_reproject = Pd
    Pv_to_reproject = Pv
    Pauli_R_to_reproject = Pauli_R
    Pauli_G_to_reproject = Pauli_G
    Pauli_B_to_reproject = Pauli_B

    # === Step 6.5: Define MLC Source CRS and Transform (Moved here to resolve UnboundLocalError) ===
    print("\nDefining MLC Source CRS and Transform...")
    # Define the appropriate UTM zone for the data's longitude.
    # Longitude for Salisbury, MD area is ~-75.6 W, which falls in UTM Zone 18N.
    src_crs = rasterio.crs.CRS.from_epsg(32618)

    # Transform the peg point from Lat/Lon to the chosen UTM zone
    transformer_ll_to_utm = Transformer.from_crs("EPSG:4326", src_crs, always_xy=True)
    peg_utm_easting, peg_utm_northing = transformer_ll_to_utm.transform(mlc_peg_lon, mlc_peg_lat)
    print(f"  - Peg Point UTM (Zone 18N): Easting={peg_utm_easting:.2f}, Northing={peg_utm_northing:.2f}")

    # Calculate the Affine transform for the MLC data in UTM
    peg_heading_rad = np.deg2rad(mlc_peg_heading)

    if mlc_look_direction.lower() == "left":
        range_heading_rad = peg_heading_rad - np.deg2rad(90)
    else: # "Right"
        range_heading_rad = peg_heading_rad + np.deg2rad(90)

    # Convert headings (CW from North) to math angles (CCW from East) for Affine matrix
    theta_col_rad = np.deg2rad(90) - range_heading_rad
    theta_row_rad = np.deg2rad(90) - peg_heading_rad

    # Affine matrix components (a, b, d, e)
    a = mlc_px_size_x * np.cos(theta_col_rad)
    d = mlc_px_size_x * np.sin(theta_col_rad)
    b = mlc_px_size_y * np.cos(theta_row_rad)
    e = mlc_px_size_y * np.sin(theta_row_rad)
    
    if e > 0:
        e = -e

    # Calculate the top-left corner (c, f) of the MLC image in UTM coordinates.
    
    # Contribution of range offset (mlc_col_addr) to Easting and Northing
    # The sign of mlc_col_addr is flipped to correct the interpretation of the
    # range offset direction from the peg point.
    offset_range_easting = -mlc_col_addr * np.cos(theta_col_rad)
    offset_range_northing = -mlc_col_addr * np.sin(theta_col_rad)

    # Contribution of azimuth offset (mlc_row_addr) to Easting and Northing
    # This is kept with its original sign.
    offset_azimuth_easting = mlc_row_addr * np.cos(theta_row_rad)
    offset_azimuth_northing = mlc_row_addr * np.sin(theta_row_rad)

    # Total offset from peg point in UTM coordinates
    total_offset_easting = offset_range_easting + offset_azimuth_easting
    total_offset_northing = offset_azimuth_northing + offset_range_northing

    # Top-left corner of the MLC image in UTM
    mlc_utm_ul_easting = peg_utm_easting + total_offset_easting
    mlc_utm_ul_northing = peg_utm_northing + total_offset_northing

    src_transform = Affine(a, b, mlc_utm_ul_easting,
                           d, e, mlc_utm_ul_northing)
    
    print(f"  - Generated MLC Source Affine Transform: {src_transform}")
    print(f"  - Generated MLC Source CRS: {src_crs.to_wkt()}")


    # === Step 6.6: Calculate Optimal Destination Grid to Avoid Clipping ===
    print("\nCalculating optimal destination grid to avoid clipping...")

    # Get the four corner points of the source MLC image in pixel coordinates
    src_corners_pixels = [
        (0, 0), (mlc_width, 0), (mlc_width, mlc_height), (0, mlc_height)
    ]

    # Transform pixel coordinates to the source UTM CRS
    src_corners_utm = [src_transform * corner for corner in src_corners_pixels]

    # Create a transformer to go from source UTM to destination Lat/Lon (GRD CRS)
    transformer_utm_to_ll = Transformer.from_crs(src_crs, grd_crs, always_xy=True)

    # Transform the UTM corner coordinates to Lat/Lon
    dst_corners_ll = [transformer_utm_to_ll.transform(e, n) for e, n in src_corners_utm]

    # Find the bounding box of the transformed corners
    dst_lons, dst_lats = zip(*dst_corners_ll)
    dst_left, dst_right = min(dst_lons), max(dst_lons)
    dst_bottom, dst_top = min(dst_lats), max(dst_lats)

    # Use the pixel size from the original GRD product for the new grid
    dst_pixel_size_x = grd_transform.a
    dst_pixel_size_y = grd_transform.e # This is negative

    # Calculate the dimensions of the new destination grid
    new_dst_width = int(np.ceil((dst_right - dst_left) / dst_pixel_size_x))
    new_dst_height = int(np.ceil((dst_top - dst_bottom) / abs(dst_pixel_size_y)))

    # Create the new Affine transform for the destination grid (north-up)
    new_dst_transform = Affine.translation(dst_left, dst_top) * Affine.scale(dst_pixel_size_x, dst_pixel_size_y)

    print(f"  - Calculated destination bounding box (Lon, Lat): Left={dst_left:.4f}, Right={dst_right:.4f}, Bottom={dst_bottom:.4f}, Top={dst_top:.4f}")
    print(f"  - New destination grid dimensions: {new_dst_width} x {new_dst_height}")
    
    # === Step 7: Reproject and Filter Components ===
    print("\nReprojecting and filtering decomposition components to the new, larger GRD grid...")

    # Check for overlap to confirm the fix
    src_bounds = array_bounds(Ps_to_reproject.shape[0], Ps_to_reproject.shape[1], src_transform)
    dst_bounds = array_bounds(grd_height, grd_width, grd_transform)
    transformer_ll_to_utm_dst = Transformer.from_crs("EPSG:4326", src_crs, always_xy=True)
    dst_utm_left, dst_utm_bottom = transformer_ll_to_utm_dst.transform(dst_bounds[0], dst_bounds[1])
    dst_utm_right, dst_utm_top = transformer_ll_to_utm_dst.transform(dst_bounds[2], dst_bounds[3])
    dst_utm_bounds = (min(dst_utm_left, dst_utm_right), min(dst_utm_bottom, dst_utm_top), 
                      max(dst_utm_left, dst_utm_right), max(dst_utm_bottom, dst_utm_top))

    print(f"  - Source Bounds (UTM): {src_bounds}")
    print(f"  - Destination Bounds (UTM for comparison): {dst_utm_bounds}")

    overlap_easting_min = max(src_bounds[0], dst_utm_bounds[0])
    overlap_easting_max = min(src_bounds[2], dst_utm_bounds[2])
    overlap_northing_min = max(src_bounds[1], dst_utm_bounds[1])
    overlap_northing_max = min(src_bounds[3], dst_utm_bounds[3])

    if overlap_easting_min < overlap_easting_max and overlap_northing_min < overlap_northing_max:
        print(f"  ✅ Overlap detected. Reprojection should succeed.")
    else:
        print("  ❌ No overlap detected. The geometric correction may still be incorrect.")

    reprojected_bands = {}
    for band_name, band_data in {
        'Ps': Ps_to_reproject, 'Pd': Pd_to_reproject, 'Pv': Pv_to_reproject,
        'Pauli_R': Pauli_R_to_reproject, 'Pauli_G': Pauli_G_to_reproject, 'Pauli_B': Pauli_B_to_reproject
    }.items():
        # MODIFICATION: Use new dimensions for the destination array
        destination_array = np.zeros((new_dst_height, new_dst_width), dtype=np.float32)
        try:
            reproject(
                source=band_data.astype(np.float32),
                destination=destination_array,
                src_transform=src_transform, 
                src_crs=src_crs, 
                # MODIFICATION: Use new destination transform
                dst_transform=new_dst_transform,
                dst_crs=grd_crs,
                resampling=Resampling.bilinear
            )
        except RasterioIOError as e:
            print(f"Error during reprojection of {band_name}: {e}")
            pass

        reprojected_bands[band_name] = refined_lee(destination_array)

    Ps_filt = reprojected_bands['Ps']
    Pd_filt = reprojected_bands['Pd']
    Pv_filt = reprojected_bands['Pv']
    Pauli_R_filt = reprojected_bands['Pauli_R']
    Pauli_G_filt = reprojected_bands['Pauli_G']
    Pauli_B_filt = reprojected_bands['Pauli_B']

    # === Step 8: Normalize and Create RGB Images ===
    print("\nNormalizing and scaling reprojected images for final output...")
    rgb_fd = np.stack([
        normalize_and_scale(Pd_filt),
        normalize_and_scale(Pv_filt),
        normalize_and_scale(Ps_filt)
    ], axis=0)

    rgb_pauli = np.stack([
        normalize_and_scale(Pauli_R_filt),
        normalize_and_scale(Pauli_G_filt),
        normalize_and_scale(Pauli_B_filt)
    ], axis=0)

    # === Step 9: Save GeoTIFFs ===
    # MODIFICATION: Use the new grid parameters in the output profile
    profile = {
        'driver': 'GTiff',
        'height': new_dst_height,
        'width': new_dst_width,
        'count': 3,
        'dtype': 'uint8',
        'crs': grd_crs,
        'transform': new_dst_transform,
        'compress': 'lzw'
    }

    print(f"\nSaving GeoTIFFs...")
    with rasterio.open(freeman_output, "w", **profile) as dst:
        dst.write(rgb_fd)
    print(f"✅ Freeman-Durden RGB saved to: {freeman_output}")

    with rasterio.open(pauli_output, "w", **profile) as dst:
        dst.write(rgb_pauli)
    print(f"✅ Pauli RGB saved to: {pauli_output}")
    
    # Return the output file paths
    return freeman_output, pauli_output


if __name__ == '__main__':
    # Call main and get the output file paths
    freeman_output_path, pauli_output_path = main()

    # === Step 10: Export Tiled Images Locally ===
    print("\n--- Starting Local Tiling Process ---")

    # Define the ROI and tile parameters based on your GEE code
    # Now, roi_bbox_for_tiling will be derived from the generated GeoTIFF's bounds
    tile_grid_size_meters = 5000 # Corresponds to GEE's 5000m grid
    output_tile_pixel_size = (256, 256) # Corresponds to GEE's 256x256 dimensions

    # Export tiles for Freeman-Durden output
    freeman_tiles_folder = "freeman_tiles"
    with rasterio.open(freeman_output_path) as src_freeman:
        bounds = src_freeman.bounds
        roi_bbox_for_tiling_freeman = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        export_tiled_image_local(freeman_output_path, freeman_tiles_folder, 
                                 roi_bbox_for_tiling_freeman, tile_grid_size_meters, 
                                 output_tile_pixel_size)

    # Export tiles for Pauli output
    pauli_tiles_folder = "pauli_tiles"
    with rasterio.open(pauli_output_path) as src_pauli:
        bounds = src_pauli.bounds
        roi_bbox_for_tiling_pauli = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        export_tiled_image_local(pauli_output_path, pauli_tiles_folder, 
                                 roi_bbox_for_tiling_pauli, tile_grid_size_meters, 
                                 output_tile_pixel_size)

    print("\n--- Local Tiling Process Complete ---")

    # === Step 11: Tiling an optical TIFF file based on existing Freeman tiles ===
    print("\n--- Starting Tiling for Optical TIFF based on Freeman Tiles ---")

    # Define the path to your optical input TIFF file
    # IMPORTANT: Replace 'path/to/your/optical_input.tif' with the actual path to your TIFF file
    optical_input_tif_path = "optical.tif" 
    optical_output_folder = "optical_tiles"

    optical_skipped_all_zero_ids = set() # Initialize an empty set for skipped IDs

    if not os.path.exists(optical_input_tif_path):
        print(f"Error: Optical input TIFF file not found at {optical_input_tif_path}. Skipping this step.")
    else:
        # Get the list of tile IDs that were actually generated for freeman_tiles
        freeman_existing_tile_ids = get_existing_tile_ids(freeman_tiles_folder)
        print(f"Found {len(freeman_existing_tile_ids)} existing tile IDs in '{freeman_tiles_folder}'.")

        # Reuse the ROI and tiling parameters from the Freeman tiling
        with rasterio.open(freeman_output_path) as src_freeman_for_optical:
            roi_bbox_for_optical_tiling = [src_freeman_for_optical.bounds.left, 
                                              src_freeman_for_optical.bounds.bottom, 
                                              src_freeman_for_optical.bounds.right, 
                                              src_freeman_for_optical.bounds.top]

        # Call export_tiled_image_local for the optical TIFF, and collect skipped IDs
        optical_tiling_stats = export_tiled_image_local(input_tif_path=optical_input_tif_path, 
                                                        export_folder=optical_output_folder, 
                                                        roi_bbox=roi_bbox_for_optical_tiling, 
                                                        tile_grid_size_meters=tile_grid_size_meters, 
                                                        output_tile_pixel_size=output_tile_pixel_size,
                                                        allowed_tile_ids=freeman_existing_tile_ids,
                                                        collect_skipped_zero_ids=True) # Set to True to collect IDs
        
        # Get the skipped IDs from the returned statistics
        optical_skipped_all_zero_ids = set(optical_tiling_stats.get("skipped_zero_ids_list", []))

    print("\n--- Optical Tiling Process Complete ---")
    
    # === Step 12: Remove corresponding tiles from Freeman and Pauli folders ===
    if optical_skipped_all_zero_ids:
        print(f"\n--- Removing {len(optical_skipped_all_zero_ids)} corresponding tiles from Freeman and Pauli folders ---")
        delete_tiles_by_ids(freeman_tiles_folder, optical_skipped_all_zero_ids)
        delete_tiles_by_ids(pauli_tiles_folder, optical_skipped_all_zero_ids)
    else:
        print("\nNo tiles were skipped in the optical tiling due to being all zeros. No corresponding tiles to remove from Freeman and Pauli folders.")


Successfully parsed .ann file.
  - MLC Dimensions: 3300 x 20308
  - GRD Dimensions: 23516 x 19393
  - GRD Upper-left corner: (-79.26222936, 35.2228176)
  - GRD Pixel size (dx, dy): (5.556e-05, -5.556e-05)
  - GRD Shear/Rotation (dx_y, dy_x): (0.0, 0.0)
  - GRD CRS: EPSG:4326
  - Generated GRD Affine Transform: | 0.00, 0.00,-79.26|
| 0.00,-0.00, 35.22|
| 0.00, 0.00, 1.00|
  - Calculated GRD Bounding Box: Left=-79.26222936, Bottom=34.14534252, Right=-77.9556804, Top=35.2228176
  - GRD Latitude Range: [34.14534252, 35.2228176]

Reading multi-look complex (.mlc) files...
  - Reading ./cpfear_13510_18065_008_180918_L090_CX_01_mlc/cpfear_13510_18065_008_180918_L090HHHH_CX_01.mlc
  - Reading ./cpfear_13510_18065_008_180918_L090_CX_01_mlc/cpfear_13510_18065_008_180918_L090HVHV_CX_01.mlc
  - Reading ./cpfear_13510_18065_008_180918_L090_CX_01_mlc/cpfear_13510_18065_008_180918_L090VVVV_CX_01.mlc
  - Reading ./cpfear_13510_18065_008_180918_L090_CX_01_mlc/cpfear_13510_18065_008_180918_L090HHHV_CX_0

In [13]:
def convert_tif_to_png(input_folder, output_folder):
    """
    Converts all GeoTIFF files in an input folder to PNG format
    and saves them in an output folder.

    Args:
        input_folder (str): Path to the folder containing input GeoTIFF files.
        output_folder (str): Path to the folder where PNG files will be saved.
    """
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    print(f"Processing folder: {input_folder}")
    print(f"Output will be saved to: {output_folder}")

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        # Check if the file is a GeoTIFF
        if filename.lower().endswith(".tif"):
            input_path = os.path.join(input_folder, filename)
            
            # Construct the output filename with a .png extension
            output_filename = os.path.splitext(filename)[0] + ".png"
            output_path = os.path.join(output_folder, output_filename)

            try:
                # Open the GeoTIFF file using rasterio
                with rasterio.open(input_path) as src:
                    # Read the first 3 bands (assuming RGB)
                    # Data is typically (bands, height, width), transpose to (height, width, bands)
                    # Convert to uint8 for Pillow
                    data = src.read()
                    
                    # Ensure we only take the first 3 bands if more exist, and handle potential alpha band
                    if src.count >= 3:
                        rgb_array = np.transpose(data[:3], (1, 2, 0)).astype(np.uint8)
                        mode = 'RGB'
                    elif src.count == 1:
                        # For single-band images, convert to grayscale RGB
                        single_band_data = data[0].astype(np.uint8)
                        rgb_array = np.stack([single_band_data, single_band_data, single_band_data], axis=-1)
                        mode = 'RGB'
                    else:
                        print(f"  Skipping {filename}: Unsupported number of bands ({src.count}).")
                        continue

                # Create a PIL Image from the NumPy array
                image = Image.fromarray(rgb_array, mode=mode)
                
                # Save the image as PNG
                image.save(output_path)
                print(f"  Converted '{filename}' to '{output_filename}'")

            except rasterio.errors.RasterioIOError as e:
                print(f"  Error reading GeoTIFF file {filename}: {e}")
            except Exception as e:
                print(f"  An unexpected error occurred while processing {filename}: {e}")

    print(f"Finished processing {input_folder}.\n")

if __name__ == "__main__":
    # Define your input and output folders
    input_folder_1 = "pauli_tiles"
    output_folder_1 = "pauli_tiles_png"

    input_folder_2 = "freeman_tiles"
    output_folder_2 = "freeman_tiles_png"

    input_folder_3 = "optical_tiles"
    output_folder_3 = "optical_tiles_png"

    # Perform the conversion for the first set of folders
    convert_tif_to_png(input_folder_1, output_folder_1)
    # Delete the input folder after conversion
    try:
        shutil.rmtree(input_folder_1)
        print(f"Successfully deleted input folder: {input_folder_1}")
    except OSError as e:
        print(f"Error deleting folder {input_folder_1}: {e}")

    # Perform the conversion for the second set of folders
    convert_tif_to_png(input_folder_2, output_folder_2)
    # Delete the input folder after conversion
    try:
        shutil.rmtree(input_folder_2)
        print(f"Successfully deleted input folder: {input_folder_2}")
    except OSError as e:
        print(f"Error deleting folder {input_folder_2}: {e}")

    # Perform the conversion for the third set of folders
    convert_tif_to_png(input_folder_3, output_folder_3)
    # Delete the input folder after conversion
    try:
        shutil.rmtree(input_folder_3)
        print(f"Successfully deleted input folder: {input_folder_3}")
    except OSError as e:
        print(f"Error deleting folder {input_folder_3}: {e}")

    print("All GeoTIFF to PNG conversions complete.")


Processing folder: pauli_tiles
Output will be saved to: pauli_tiles_png
  Converted 'tile_231.tif' to 'tile_231.png'
  Converted 'tile_184.tif' to 'tile_184.png'
  Converted 'tile_345.tif' to 'tile_345.png'
  Converted 'tile_44.tif' to 'tile_44.png'
  Converted 'tile_437.tif' to 'tile_437.png'
  Converted 'tile_386.tif' to 'tile_386.png'
  Converted 'tile_93.tif' to 'tile_93.png'
  Converted 'tile_392.tif' to 'tile_392.png'
  Converted 'tile_92.tif' to 'tile_92.png'
  Converted 'tile_387.tif' to 'tile_387.png'
  Converted 'tile_436.tif' to 'tile_436.png'
  Converted 'tile_45.tif' to 'tile_45.png'
  Converted 'tile_344.tif' to 'tile_344.png'
  Converted 'tile_185.tif' to 'tile_185.png'
  Converted 'tile_230.tif' to 'tile_230.png'
  Converted 'tile_226.tif' to 'tile_226.png'
  Converted 'tile_232.tif' to 'tile_232.png'
  Converted 'tile_187.tif' to 'tile_187.png'
  Converted 'tile_434.tif' to 'tile_434.png'
  Converted 'tile_346.tif' to 'tile_346.png'
  Converted 'tile_90.tif' to 'tile_9

/var/folders/w3/skv5rsks5d98pcs6hxw0gr_80000gn/T/ipykernel_94811/696001490.py:47: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  image = Image.fromarray(rgb_array, mode=mode)


  Converted 'tile_43.tif' to 'tile_43.png'
  Converted 'tile_342.tif' to 'tile_342.png'
  Converted 'tile_183.tif' to 'tile_183.png'
  Converted 'tile_140.tif' to 'tile_140.png'
  Converted 'tile_208.tif' to 'tile_208.png'
  Converted 'tile_181.tif' to 'tile_181.png'
  Converted 'tile_432.tif' to 'tile_432.png'
  Converted 'tile_368.tif' to 'tile_368.png'
  Converted 'tile_69.tif' to 'tile_69.png'
  Converted 'tile_68.tif' to 'tile_68.png'
  Converted 'tile_369.tif' to 'tile_369.png'
  Converted 'tile_433.tif' to 'tile_433.png'
  Converted 'tile_341.tif' to 'tile_341.png'
  Converted 'tile_209.tif' to 'tile_209.png'
  Converted 'tile_252.tif' to 'tile_252.png'
  Converted 'tile_118.tif' to 'tile_118.png'
  Converted 'tile_483.tif' to 'tile_483.png'
  Converted 'tile_482.tif' to 'tile_482.png'
  Converted 'tile_253.tif' to 'tile_253.png'
  Converted 'tile_251.tif' to 'tile_251.png'
  Converted 'tile_319.tif' to 'tile_319.png'
  Converted 'tile_457.tif' to 'tile_457.png'
  Converted 'til